In [1]:
import numpy as np
import librosa
import torch
import laion_clap
from extraction.vgg_sound import *
from audioldm import image_to_audio, build_model, clap_to_audio
import torch
import torchaudio
from transformers import AutoProcessor, AutoModel
import soundfile as sf 



/home/ubuntu/miniconda3/envs/v2a-mapper/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ubuntu/miniconda3/envs/v2a-mapper/lib/python3.11/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/home/ubuntu/miniconda3/envs/v2a-mapper/lib/python3.11/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [2]:
from audioldm.clap.encoders import CLAPAudioEmbeddingClassifierFreev2
CLAP = CLAPAudioEmbeddingClassifierFreev2(
    key='waveform',
    pretrained_path="/home/ubuntu/project/v2a-mapper/pretrain/clap_htsat_tiny.pt",
    sampling_rate=16000,
    embed_mode="audio",
    amodel="HTSAT-tiny"
)

DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /roberta-base/resolve/main/vocab.json HTTP/1.1" 200 0


DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /roberta-base/resolve/main/vocab.json HTTP/1.1" 200 0
INFO:root:Loading HTSAT-tiny model config.
/home/ubuntu/miniconda3/envs/v2a-mapper/lib/python3.11/site-packages/torchlibrosa/stft.py:193: FutureWarning: Pass size=1024 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  fft_window = librosa.util.pad_center(fft_window, n_fft)
/home/ubuntu/miniconda3/envs/v2a-mapper/lib/python3.11/site-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /roberta-base/resolve/main/config.json HTTP/1.1" 200 0
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel

In [4]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

root = "/mnt/new_volume2/vgg_sound_emb"
partition = "train"
data_dir = f"{root}/{partition}"

class LargeVideoDataset(Dataset):
    def __init__(self, data_dir, subset_ratio = 0.2, transform=None):
        """
        root_dir: 保存所有 .pth 文件的目录，每个文件对应一个 sample。
        transform: 如果需要对数据做预处理，可在这里传入。
        """
        super().__init__()
        # 仅收集当前目录下所有的 pth 文件列表
        file_list = []

        for root, dirs, files in os.walk(data_dir):
            for file in files:
                if file.endswith(".pth"):
                    file_list.append(os.path.join(root, file))

        # 仅使用前 20% 的数据
        num_samples = int(len(file_list) * subset_ratio)

        self.file_paths = sorted(file_list)[:num_samples]
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        # 在这里按需读取，而不是一次性加载全部
        pth_path = self.file_paths[idx]
        sample_data = torch.load(pth_path)  
        clip_feat = sample_data['clip_features']  # (64, 512)
        clap_feat = sample_data['clap_features']  # (1, 512)

        if self.transform:
            clip_feat, clap_feat = self.transform((clip_feat, clap_feat))

        return clip_feat, clap_feat

In [6]:
video_dataset = LargeVideoDataset(data_dir, subset_ratio=0.2)

In [15]:
video_dataset[0][1].shape

torch.Size([1, 512])

In [5]:
# vgg_dataset = VGGSound(root="/home/ubuntu/project/subdata/video",
#                            csv_path="/home/ubuntu/project/subdata/train_subset.csv",
#                            sample_rate =  16000)


# data = vgg_dataset[7]

In [4]:
data

{'id': '1JWJSFMGrN4_000030.mp4',
 'caption': 'male singing',
 'audio': tensor([-0.0032,  0.0149,  0.0367,  ...,  0.0535,  0.0584,  0.0576]),
 'clip_video': Image([[[[0.0627, 0.0627, 0.0627,  ..., 0.0627, 0.0627, 0.0627],
          [0.0627, 0.0627, 0.0627,  ..., 0.0627, 0.0627, 0.0627],
          [0.0627, 0.0627, 0.0627,  ..., 0.0627, 0.0627, 0.0627],
          ...,
          [0.0627, 0.0627, 0.0627,  ..., 0.0627, 0.0627, 0.0627],
          [0.0627, 0.0627, 0.0627,  ..., 0.0627, 0.0627, 0.0627],
          [0.0627, 0.0627, 0.0627,  ..., 0.0627, 0.0627, 0.0627]],
 
         [[0.0627, 0.0627, 0.0627,  ..., 0.0627, 0.0627, 0.0627],
          [0.0627, 0.0627, 0.0627,  ..., 0.0627, 0.0627, 0.0627],
          [0.0627, 0.0627, 0.0627,  ..., 0.0627, 0.0627, 0.0627],
          ...,
          [0.0627, 0.0627, 0.0627,  ..., 0.0627, 0.0627, 0.0627],
          [0.0627, 0.0627, 0.0627,  ..., 0.0627, 0.0627, 0.0627],
          [0.0627, 0.0627, 0.0627,  ..., 0.0627, 0.0627, 0.0627]],
 
         [[0.0627

In [6]:
# audio_bs = data['audio'].unsqueeze(0).unsqueeze(0).shape

In [ ]:
# embed = CLAP.forward(data['audio'].unsqueeze(0).to('cuda'))

/home/ubuntu/miniconda3/envs/v2a-mapper/lib/python3.11/site-packages/torchaudio/transforms/_transforms.py:580: UserWarning: Argument 'onesided' has been deprecated and has no influence on the behavior of this module.
  warnings.warn(


In [9]:
# embed[0]

In [8]:

MODEL_NAME = "audioldm-s-full-v2"
audioldm=build_model(model_name=MODEL_NAME)



Load AudioLDM: %s audioldm-s-full-v2
DiffusionWrapper has 185.04 M params.


/home/ubuntu/miniconda3/envs/v2a-mapper/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/home/ubuntu/miniconda3/envs/v2a-mapper/lib/python3.11/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /roberta-base/resolve/main/vocab.json HTTP/1.1" 200 0
INFO:root:Loading HTSAT-tiny model config.
/home/ubuntu/miniconda3/envs/v2a-mapper/lib/python3.11/site-packages/torchlibrosa/stft.py:193: FutureWarning: Pass size=1024 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  fft_window = librosa.util.pad_center(fft_windo

In [ ]:

def image2audio(file_name, images, duration=10, guidance_scale=2.5, random_seed=42, n_candidates=3):
    waveform = clap_to_audio(
        latent_diffusion=audioldm,
        clap_feat=images,
        seed=random_seed,
        duration=duration,
        guidance_scale=guidance_scale,
        n_candidate_gen_per_text=int(n_candidates),
    )  # [bs, 1, samples]

    for i, wave in enumerate(waveform):
        filename = f"../output/{file_name}.wav"
        sf.write(filename, wave[0], 16000, 'PCM_16') 

    return waveform

image2audio(
    file_name="test2",
    images=video_dataset[2][1].to('cuda:0'),
    duration=10,
    guidance_scale=2.5,
    random_seed=42,
    n_candidates=3
)

In [18]:
image2audio(
    file_name="test2",
    images=video_dataset[2][1].to('cuda:0'),
    duration=10,
    guidance_scale=2.5,
    random_seed=42,
    n_candidates=3
)

batch shape:  [tensor([[-1.8063e-02,  4.8113e-02,  3.5327e-02,  1.9750e-02, -1.2734e-02,
         -2.0392e-02, -1.8764e-02,  2.5611e-02, -3.5815e-03, -9.2289e-03,
          3.7888e-03, -7.9979e-02,  2.4839e-02,  9.6638e-02,  2.1745e-02,
         -5.8274e-02, -9.0003e-03, -2.5673e-03,  5.9004e-02,  2.8808e-02,
          4.1845e-02, -1.0983e-02, -4.7954e-02,  3.9691e-02, -9.9765e-03,
         -1.9808e-02, -3.6131e-02, -1.7208e-02,  2.9788e-03,  2.3228e-02,
          6.1867e-02,  3.1000e-02, -3.6792e-02, -2.0753e-02, -2.7919e-02,
          3.3872e-02, -4.1604e-02, -4.5168e-02, -1.0017e-03, -5.2476e-02,
          2.5796e-02,  9.3767e-03, -1.5496e-02,  6.9663e-02, -1.6264e-02,
         -4.9651e-02,  2.8183e-02, -2.0987e-02,  1.3949e-02,  2.7978e-02,
         -1.3387e-02, -6.5049e-02, -6.1874e-02,  5.2723e-03,  8.7065e-03,
         -7.6076e-02,  3.0232e-02, -8.2282e-02, -3.7384e-02,  1.6844e-02,
          2.7681e-02, -3.3685e-02,  2.1811e-02,  2.1942e-02, -7.0906e-04,
          6.8498e-02,  

DDIM Sampler:   0%|          | 0/200 [00:00<?, ?it/s]

DDIM Sampler: 100%|██████████| 200/200 [00:17<00:00, 11.56it/s]


array([[[0.00018654, 0.00015933, 0.00016566, ..., 0.00017977,
         0.00019992, 0.00014191]]], dtype=float32)

In [ ]:
# import torch
# import torchaudio
# from transformers import AutoProcessor, AutoModel

# # Load processor and model
# processor = AutoProcessor.from_pretrained("laion/clap-htsat-unfused")
# model = AutoModel.from_pretrained("laion/clap-htsat-unfused")

In [3]:
from extraction.vgg_sound import *

In [4]:
vgg_dataset = VGGSound(root="/home/ubuntu/project/subdata/video",
                           csv_path="/home/ubuntu/project/subdata/train_subset.csv")

data = vgg_dataset[0]

# for data in vgg_dataset:
#     # print(data['id'])
#     # print(data['audio'].shape)
#     # print(data['video'].shape)
#     # print(data['video'].shape)
#     # if data['id'] == '1msyXyqRvpY_000000.mp4':
#     #     print(data['audio'].shape)
#     break

INFO:root:17 videos found in /home/ubuntu/project/subdata/video
INFO:root:17 videos found in /home/ubuntu/project/subdata/train_subset.csv
INFO:root:48 videos missing in /home/ubuntu/project/subdata/video


In [5]:
data['clip_video']

Image([[[[0.1294, 0.1294, 0.1333,  ..., 0.3961, 0.3059, 0.2784],
         [0.1255, 0.1255, 0.1294,  ..., 0.3961, 0.3059, 0.2784],
         [0.1255, 0.1255, 0.1294,  ..., 0.3843, 0.2980, 0.2706],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.1020, 0.1020, 0.1020],
         [0.0000, 0.0000, 0.0000,  ..., 0.1020, 0.1020, 0.1020],
         [0.0000, 0.0000, 0.0000,  ..., 0.0980, 0.1020, 0.1020]],

        [[0.1451, 0.1451, 0.1490,  ..., 0.4353, 0.3569, 0.3255],
         [0.1412, 0.1412, 0.1451,  ..., 0.4353, 0.3569, 0.3255],
         [0.1412, 0.1412, 0.1451,  ..., 0.4314, 0.3569, 0.3255],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0784, 0.0784, 0.0784],
         [0.0000, 0.0000, 0.0000,  ..., 0.0784, 0.0784, 0.0784],
         [0.0000, 0.0000, 0.0000,  ..., 0.0745, 0.0784, 0.0784]],

        [[0.1765, 0.1765, 0.1804,  ..., 0.4275, 0.3725, 0.3765],
         [0.1725, 0.1725, 0.1765,  ..., 0.4275, 0.3725, 0.3765],
         [0.1686, 0.1686, 0.1765,  ..., 0.4118, 0.3647, 0.

In [14]:

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
model.eval()

# # Load audio file
# audio_path = "input_audio.wav"
# waveform, sr = torchaudio.load(audio_path)
waveform = data['audio']

# Resample to 48kHz (required by CLAP)
# if sr != 48000:
# resampler = torchaudio.transforms.Resample(16000, 48000)
# waveform = resampler(waveform)

# Convert to mono
# if waveform.shape[0] > 1:
#     waveform = torch.mean(waveform, dim=0, keepdim=True)

# Prepare inputs for the model
inputs = processor(audios=waveform, sampling_rate=48000, return_tensors="pt")
inputs = {k: v.to(device) for k, v in inputs.items()}

# Extract audio embeddings
with torch.no_grad():
    outputs = model.get_audio_features(**inputs)  # shape: [batch, feature_dim]

audio_embed = outputs  # shape: [1, 512]


In [16]:
audio_embed.shape

torch.Size([1, 512])

In [ ]:

# # quantization
# def int16_to_float32(x):
#     return (x / 32767.0).astype(np.float32)


# def float32_to_int16(x):
#     x = np.clip(x, a_min=-1., a_max=1.)
#     return (x * 32767.).astype(np.int16)

# model = laion_clap.CLAP_Module(enable_fusion=False)
# model.load_ckpt() # download the default pretrained checkpoint.


/home/ubuntu/miniconda3/envs/v2a-mapper/lib/python3.11/site-packages/torchlibrosa/stft.py:193: FutureWarning: Pass size=1024 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  fft_window = librosa.util.pad_center(fft_window, n_fft)
/home/ubuntu/miniconda3/envs/v2a-mapper/lib/python3.11/site-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializ

UnpicklingError: Weights only load failed. This file can still be loaded, to do so you have two options, [1mdo those steps only if you trust the source of the checkpoint[0m. 
	(1) In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
	(2) Alternatively, to load with `weights_only=True` please check the recommended steps in the following error message.
	WeightsUnpickler error: Unsupported global: GLOBAL numpy.core.multiarray.scalar was not an allowed global by default. Please use `torch.serialization.add_safe_globals([scalar])` or the `torch.serialization.safe_globals([scalar])` context manager to allowlist this global if you trust this class/function.

Check the documentation of torch.load to learn more about types accepted by default with weigh

In [23]:
data['clip_video']

Image([[[[0.1294, 0.1294, 0.1333,  ..., 0.3961, 0.3059, 0.2784],
         [0.1255, 0.1255, 0.1294,  ..., 0.3961, 0.3059, 0.2784],
         [0.1255, 0.1255, 0.1294,  ..., 0.3843, 0.2980, 0.2706],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.1020, 0.1020, 0.1020],
         [0.0000, 0.0000, 0.0000,  ..., 0.1020, 0.1020, 0.1020],
         [0.0000, 0.0000, 0.0000,  ..., 0.0980, 0.1020, 0.1020]],

        [[0.1451, 0.1451, 0.1490,  ..., 0.4353, 0.3569, 0.3255],
         [0.1412, 0.1412, 0.1451,  ..., 0.4353, 0.3569, 0.3255],
         [0.1412, 0.1412, 0.1451,  ..., 0.4314, 0.3569, 0.3255],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0784, 0.0784, 0.0784],
         [0.0000, 0.0000, 0.0000,  ..., 0.0784, 0.0784, 0.0784],
         [0.0000, 0.0000, 0.0000,  ..., 0.0745, 0.0784, 0.0784]],

        [[0.1765, 0.1765, 0.1804,  ..., 0.4275, 0.3725, 0.3765],
         [0.1725, 0.1725, 0.1765,  ..., 0.4275, 0.3725, 0.3765],
         [0.1686, 0.1686, 0.1765,  ..., 0.4118, 0.3647, 0.

In [17]:
img = data['clip_video'][0, :, :, :]

In [22]:
data['clip_video']

Image([[[[0.1294, 0.1294, 0.1333,  ..., 0.3961, 0.3059, 0.2784],
         [0.1255, 0.1255, 0.1294,  ..., 0.3961, 0.3059, 0.2784],
         [0.1255, 0.1255, 0.1294,  ..., 0.3843, 0.2980, 0.2706],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.1020, 0.1020, 0.1020],
         [0.0000, 0.0000, 0.0000,  ..., 0.1020, 0.1020, 0.1020],
         [0.0000, 0.0000, 0.0000,  ..., 0.0980, 0.1020, 0.1020]],

        [[0.1451, 0.1451, 0.1490,  ..., 0.4353, 0.3569, 0.3255],
         [0.1412, 0.1412, 0.1451,  ..., 0.4353, 0.3569, 0.3255],
         [0.1412, 0.1412, 0.1451,  ..., 0.4314, 0.3569, 0.3255],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0784, 0.0784, 0.0784],
         [0.0000, 0.0000, 0.0000,  ..., 0.0784, 0.0784, 0.0784],
         [0.0000, 0.0000, 0.0000,  ..., 0.0745, 0.0784, 0.0784]],

        [[0.1765, 0.1765, 0.1804,  ..., 0.4275, 0.3725, 0.3765],
         [0.1725, 0.1725, 0.1765,  ..., 0.4275, 0.3725, 0.3765],
         [0.1686, 0.1686, 0.1765,  ..., 0.4118, 0.3647, 0.

In [16]:
img.shape

(3, 224, 224)

In [ ]:
# Directly get audio embeddings from audio files
audio_file = [
    '/home/data/test_clap_short.wav',
    '/home/data/test_clap_long.wav'
]
audio_embed = model.get_audio_embedding_from_filelist(x = audio_file, use_tensor=False)
print(audio_embed[:,-20:])
print(audio_embed.shape)

# Get audio embeddings from audio data
audio_data, _ = librosa.load('/home/data/test_clap_short.wav', sr=48000) # sample rate should be 48000
audio_data = audio_data.reshape(1, -1) # Make it (1,T) or (N,T)
audio_embed = model.get_audio_embedding_from_data(x = audio_data, use_tensor=False)
print(audio_embed[:,-20:])
print(audio_embed.shape)

# Directly get audio embeddings from audio files, but return torch tensor
audio_file = [
    '/home/data/test_clap_short.wav',
    '/home/data/test_clap_long.wav'
]
audio_embed = model.get_audio_embedding_from_filelist(x = audio_file, use_tensor=True)
print(audio_embed[:,-20:])
print(audio_embed.shape)

# Get audio embeddings from audio data
audio_data, _ = librosa.load('/home/data/test_clap_short.wav', sr=48000) # sample rate should be 48000
audio_data = audio_data.reshape(1, -1) # Make it (1,T) or (N,T)
audio_data = torch.from_numpy(int16_to_float32(float32_to_int16(audio_data))).float() # quantize before send it in to the model
audio_embed = model.get_audio_embedding_from_data(x = audio_data, use_tensor=True)
print(audio_embed[:,-20:])
print(audio_embed.shape)

# Get text embedings from texts:
text_data = ["I love the contrastive learning", "I love the pretrain model"] 
text_embed = model.get_text_embedding(text_data)
print(text_embed)
print(text_embed.shape)

# Get text embedings from texts, but return torch tensor:
text_data = ["I love the contrastive learning", "I love the pretrain model"] 
text_embed = model.get_text_embedding(text_data, use_tensor=True)
print(text_embed)
print(text_embed.shape)


In [ ]:
from audioldm import image_to_audio, build_model
MODEL_NAME = "audioldm-s-full-v2"
audioldm=build_model(model_name=MODEL_NAME)

def image2audio(file_name, images, duration=10, guidance_scale=2.5, random_seed=42, n_candidates=3):
    waveform = image_to_audio(
        latent_diffusion=audioldm,
        images=images,
        seed=random_seed,
        duration=duration,
        guidance_scale=guidance_scale,
        n_candidate_gen_per_text=int(n_candidates),
    )  # [bs, 1, samples]

    for i, wave in enumerate(waveform):
        filename = f"../output/{file_name}.wav"
        sf.write(filename, wave[0], 16000, 'PCM_16') 

    return waveform

image2audio(
    file_name="test",
    images=data['clip_video'],
    duration=10,
    guidance_scale=2.5,
    random_seed=42,
    n_candidates=3
)

In [7]:
MODEL_NAME = "audioldm-s-full-v2"
audioldm=build_model(model_name=MODEL_NAME)

Load AudioLDM: %s audioldm-s-full-v2
DiffusionWrapper has 185.04 M params.


/home/ubuntu/miniconda3/envs/v2a-mapper/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/home/ubuntu/miniconda3/envs/v2a-mapper/lib/python3.11/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /roberta-base/resolve/main/vocab.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /roberta-base/resolve/main/vocab.json HTTP/1.1" 200 0
INFO:root:Loading HTSAT-tiny model config.
/home/ubuntu/miniconda3/envs/v2a-mapper/lib/python3.11/site-packages/torchlibrosa/stft.py:193: FutureWarning: Pass size=1024 as keyword args. From version

In [8]:
import soundfile as sf 

In [9]:
def image2audio(file_name, images, duration=10, guidance_scale=2.5, random_seed=42, n_candidates=3):
    waveform = image_to_audio(
        latent_diffusion=audioldm,
        images=images,
        seed=random_seed,
        duration=duration,
        guidance_scale=guidance_scale,
        n_candidate_gen_per_text=int(n_candidates),
    )  # [bs, 1, samples]

    for i, wave in enumerate(waveform):
        filename = f"../output/{file_name}.wav"
        sf.write(filename, wave[0], 16000, 'PCM_16') 

    return waveform

In [10]:
image2audio(
    file_name="test",
    images=data['clip_video'],
    duration=10,
    guidance_scale=2.5,
    random_seed=42,
    n_candidates=3
)

Generate audio using image Image([[[[0.1294, 0.1294, 0.1333,  ..., 0.3961, 0.3059, 0.2784],
         [0.1255, 0.1255, 0.1294,  ..., 0.3961, 0.3059, 0.2784],
         [0.1255, 0.1255, 0.1294,  ..., 0.3843, 0.2980, 0.2706],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.1020, 0.1020, 0.1020],
         [0.0000, 0.0000, 0.0000,  ..., 0.1020, 0.1020, 0.1020],
         [0.0000, 0.0000, 0.0000,  ..., 0.0980, 0.1020, 0.1020]],

        [[0.1451, 0.1451, 0.1490,  ..., 0.4353, 0.3569, 0.3255],
         [0.1412, 0.1412, 0.1451,  ..., 0.4353, 0.3569, 0.3255],
         [0.1412, 0.1412, 0.1451,  ..., 0.4314, 0.3569, 0.3255],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0784, 0.0784, 0.0784],
         [0.0000, 0.0000, 0.0000,  ..., 0.0784, 0.0784, 0.0784],
         [0.0000, 0.0000, 0.0000,  ..., 0.0745, 0.0784, 0.0784]],

        [[0.1765, 0.1765, 0.1804,  ..., 0.4275, 0.3725, 0.3765],
         [0.1725, 0.1725, 0.1765,  ..., 0.4275, 0.3725, 0.3765],
         [0.1686, 0.1686, 0.176

RuntimeError: Could not infer dtype of NoneType

In [31]:
audioldm.first_stage_model

AutoencoderKL(
  (encoder): Encoder(
    (conv_in): Conv2d(1, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (down): ModuleList(
      (0): Module(
        (block): ModuleList(
          (0-1): 2 x ResnetBlock(
            (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
            (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (norm2): GroupNorm(32, 128, eps=1e-06, affine=True)
            (dropout): Dropout(p=0.0, inplace=False)
            (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          )
        )
        (attn): ModuleList()
        (downsample): Downsample(
          (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2))
        )
      )
      (1): Module(
        (block): ModuleList(
          (0): ResnetBlock(
            (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
            (conv1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (no

In [2]:
from audioldm.clap.encoders import CLAPAudioEmbeddingClassifierFreev2
CLAP = CLAPAudioEmbeddingClassifierFreev2(
    key='waveform',
    pretrained_path="/home/ubuntu/project/v2a-mapper/pretrain/clap_htsat_tiny.pt",
    sampling_rate=16000,
    embed_mode="audio",
    amodel="HTSAT-tiny"
)

DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /roberta-base/resolve/main/vocab.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /roberta-base/resolve/main/vocab.json HTTP/1.1" 200 0
INFO:root:Loading HTSAT-tiny model config.
/home/ubuntu/miniconda3/envs/v2a-mapper/lib/python3.11/site-packages/torchlibrosa/stft.py:193: FutureWarning: Pass size=1024 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  fft_window = librosa.util.pad_center(fft_window, n_fft)
/home/ubuntu/miniconda3/envs/v2a-mapper/lib/python3.11/site-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /roberta-base/resolve/main/config.js

In [5]:
from audioldm.utils import default_audioldm_config
config = default_audioldm_config()
config

{'wave_file_save_path': './output',
 'id': {'version': 'v1',
  'name': 'default',
  'root': '/mnt/fast/nobackup/users/hl01486/projects/general_audio_generation/AudioLDM-python/config/default/latent_diffusion.yaml'},
 'preprocessing': {'audio': {'sampling_rate': 16000, 'max_wav_value': 32768},
  'stft': {'filter_length': 1024, 'hop_length': 160, 'win_length': 1024},
  'mel': {'n_mel_channels': 64,
   'mel_fmin': 0,
   'mel_fmax': 8000,
   'freqm': 0,
   'timem': 0,
   'blur': False,
   'mean': -4.63,
   'std': 2.74,
   'target_length': 1024}},
 'model': {'device': 'cuda',
  'target': 'audioldm.pipline.LatentDiffusion',
  'params': {'base_learning_rate': 5e-06,
   'linear_start': 0.0015,
   'linear_end': 0.0195,
   'num_timesteps_cond': 1,
   'log_every_t': 200,
   'timesteps': 1000,
   'first_stage_key': 'fbank',
   'cond_stage_key': 'waveform',
   'latent_t_size': 256,
   'latent_f_size': 16,
   'channels': 8,
   'cond_stage_trainable': True,
   'conditioning_key': 'film',
   'monitor'